In [1]:
#essential imports
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize

#scraping imports
import requests
from bs4 import BeautifulSoup

#plotting imports
%matplotlib inline
import matplotlib.pyplot as plt

#String matching
import re

#date
import datetime as dt

# Matching movies and dead actors/directors

## 1) Clean the meta_movies data generated with Amazon API

In [4]:
meta_movies_TV = pd.read_csv('DATA/metadata_processed/meta_Movies_and_TV_processed.csv', low_memory=False)

In [5]:
meta_movies_TV.head()

,Unnamed: 0,asin,categories,description,title,salesRank,actors,directors,creators,authors,model,artist
0,0,0000143561,"[['Movies & TV', 'Movies']]","3Pack DVD set - Italian Classics, Parties and ...","Everyday Italian (with Giada de Laurentiis), V...",{'Movies & TV': 376041},[],[],[],[],NaN,NaN
1,1,0000589012,"[['Movies & TV', 'Movies']]",NaN,Why Don't They Just Quit? DVD Roundtable Discu...,{'Movies & TV': 1084845},[],[],[],['Joe Herzanek'],NaN,NaN
2,2,0000695009,"[['Movies & TV', 'Movies']]",NaN,Understanding Seizures and Epilepsy DVD,{'Movies & TV': 1022732},[],[],[],['Epilepsy Foundation'],NaN,NaN
3,3,000107461X,"[['Movies & TV', 'Movies']]",NaN,Live in Houston [VHS],{'Movies & TV': 954116},['Douglas Miller'],[],"[('Joe Thomas', 'Producer'), ('Joseph Williams...",[],NaN,NaN
4,4,0000143529,"[['Movies & TV', 'Movies']]",Disc 1: Flour Power (Scones; Shortcakes; South...,My Fair Pastry (Good Eats Vol. 9),{'Movies & TV': 463562},['Alton Brown'],[],[],[],NaN,NaN


In [6]:
meta_movies_TV.columns

Index(['Unnamed: 0', 'asin', 'categories', 'description', 'title', 'salesRank',
       'actors', 'directors', 'creators', 'authors', 'model', 'artist'],
      dtype='object')

In [45]:
clean_MM_TV = meta_movies_TV.drop(['Unnamed: 0','actors','directors','categories'],axis=1)

def clean_serie(series_):
    return series_.str.replace("[", "").str.replace("]","").str.replace("'","").str.replace(".","").str.replace(", ",",").str.replace('"','')
    #return list(series_)

clean_MM_TV['actors'] = clean_serie(meta_movies_TV['actors'])
clean_MM_TV['directors'] = clean_serie(meta_movies_TV['directors'])
clean_MM_TV['categories'] = clean_serie(meta_movies_TV['categories'])

In [46]:
clean_MM_TV

,asin,description,title,salesRank,creators,authors,model,artist,actors,directors,categories
0,0000143561,"3Pack DVD set - Italian Classics, Parties and ...","Everyday Italian (with Giada de Laurentiis), V...",{'Movies & TV': 376041},[],[],NaN,NaN,,,"Movies & TV,Movies"
1,0000589012,NaN,Why Don't They Just Quit? DVD Roundtable Discu...,{'Movies & TV': 1084845},[],['Joe Herzanek'],NaN,NaN,,,"Movies & TV,Movies"
2,0000695009,NaN,Understanding Seizures and Epilepsy DVD,{'Movies & TV': 1022732},[],['Epilepsy Foundation'],NaN,NaN,,,"Movies & TV,Movies"
3,000107461X,NaN,Live in Houston [VHS],{'Movies & TV': 954116},"[('Joe Thomas', 'Producer'), ('Joseph Williams...",[],NaN,NaN,Douglas Miller,,"Movies & TV,Movies"
4,0000143529,Disc 1: Flour Power (Scones; Shortcakes; South...,My Fair Pastry (Good Eats Vol. 9),{'Movies & TV': 463562},[],[],NaN,NaN,Alton Brown,,"Movies & TV,Movies"
5,0000143502,Rise and Swine (Good Eats Vol. 7) includes bon...,Rise and Swine (Good Eats Vol. 7),{'Movies & TV': 467172},[],[],NaN,NaN,Alton Brown,,"Movies & TV,Movies"
6,0000143588,Barefoot Contessa Volume 2: On these three dis...,"Barefoot Contessa (with Ina Garten), Entertain...",{'Movies & TV': 309405},[],[],NaN,NaN,Ina Garten,,"Movies & TV,Movies"
7,0001517791,Praise Aerobics - A low-intensity/high-intesit...,Praise Aerobics [VHS],{'Movies & TV': 557724},"[('Integrity Music Fitness', 'Recorder')]",[],NaN,NaN,,,"Movies & TV,Movies"
8,0001527665,NaN,Peace Child [VHS],{'Movies & TV': 734524},[],[],NaN,NaN,,,"Movies & TV,Movies"
9,0001516035,Worship with Don Moen [VHS],Worship with Don Moen [VHS],{'Movies & TV': 435882},"[('Don Moen', 'Recorder')]",[],NaN,NaN,,,"Movies & TV,Movies"


## 2) Cleaning and filtering the dead names data

In [47]:
deaths = pd.read_csv('DATA/deaths.csv', low_memory=False, encoding="ISO-8859-1")

In [48]:
deaths.head()

,Name,Birth Date,Death Date,Description,Actor,Author,Musician
0,Jack Weston,1924,1996-05-03,", american actor",True,False,False
1,John Beradino,1917,1996-05-19,", american baseball player and actor",True,False,False
2,Jon Pertwee,1919,1996-05-20,", british actor",True,False,False
3,Paul Delph,1957,1996-05-21,", american musician and producer",False,False,True
4,Lash LaRue,1917,1996-05-21,", american actor",True,False,False


In [49]:
dead_actors = deaths[deaths['Actor']==True].reset_index()
dead_actors.head()

,index,Name,Birth Date,Death Date,Description,Actor,Author,Musician
0,0,Jack Weston,1924,1996-05-03,", american actor",True,False,False
1,1,John Beradino,1917,1996-05-19,", american baseball player and actor",True,False,False
2,2,Jon Pertwee,1919,1996-05-20,", british actor",True,False,False
3,4,Lash LaRue,1917,1996-05-21,", american actor",True,False,False
4,6,Enrique Álvarez Félix,1934,1996-05-24,", mexican actor",True,False,False


## 3) Build controls list

In [50]:
actors_products = clean_MM_TV[clean_MM_TV['actors'].isnull()==False]
actors_products = actors_products[actors_products["actors"].isin([""])==False]

actors_names = [troup.split(',') for troup in list(actors_products['actors'])]
all_actors = list(set(np.concatenate(actors_names)))

In [51]:
all_actors

['',
 'Scott Jones',
 'Minor Watson',
 'Caroline Sandry',
 'Brad Sherwood',
 'Page Clements',
 'Takako Uehara',
 'Geoff Meed',
 'Miki Manojlovi',
 'Andrea Fears',
 'Ally Dunne',
 'Paulie Z',
 'Nadine Chalmers',
 'John Kelly',
 'Robert Randall',
 'Jessica English',
 'Hugh Masekela',
 'Diana Golden',
 'David Eckstein',
 'Theodor Guschulbauer',
 'Alan  Peters Orchestra',
 'Elwood Babbitt',
 'Susan Egan',
 'Jin-yeong Jeong',
 'Wiener Philharmoniker',
 'Kuntschew',
 'Dani Apple',
 'Kenneth Michael Terry',
 'Gang Marcela',
 'Wali Zarif',
 'Teresa Maldonado Marchok-Physical Therapist',
 'Olivier',
 'Cello Suites',
 'Cho Hyun-chul',
 'Monem Salam',
 'Nishijima Hidetoshi',
 'Kathryn Kvols',
 'Pete Freezin',
 'Cary Grant',
 'Bella Diona',
 'GB Jones',
 'Margaret Johnston',
 'Peter Simonischek',
 'Arun Nalwade',
 'Susan Sennett',
 'Ramón Agirre',
 'Jesus Guzman',
 'Charles Mcgraw',
 'William Shimell',
 'Rosci Diaz',
 'Daniel Amalm',
 'Adama Niane',
 'Wanda Collins',
 'Etta Baker',
 'Mino Manni',


In [52]:
all_controls = pd.DataFrame(all_actors, columns = ['name'])
all_controls = all_controls[all_controls['name'].isin(list(dead_actors['Name']))==False].reset_index()

In [53]:
len(all_controls), len(dead_actors)

(145217, 639)

In [54]:
controls = pd.DataFrame([all_controls['name'].get_value(idx) for idx
                        in np.round(np.linspace(0,131111,941))],
                        columns =['Name'])

## 4) Matching dead actors and meta data

In [55]:
pd.options.mode.chained_assignment = None 
meta_dead_actors = pd.DataFrame()

for idx, actor in enumerate(list(dead_actors['Name'])):
    
    match = pd.DataFrame()
    match = actors_products[actors_products['actors'].str.contains(actor)==True].drop('actors',axis=1).drop('directors',axis=1)
    match['name'] = actor
    match['death date'] = dead_actors['Death Date'].get_value(idx)
    meta_dead_actors = meta_dead_actors.append(match)
    
    if idx %100 == 0: print('passing the {}th actor'.format(idx))
        
    

passing the 0th actor


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  import sys


passing the 100th actor
passing the 200th actor
passing the 300th actor
passing the 400th actor
passing the 500th actor
passing the 600th actor


In [19]:
meta_dead_actors.head()

,asin,description,title,price,salesRank,imUrl,related,brand,categories,name,death date
3487,0792840461,NaN,The Thomas Crown Affair,13.50,{'Movies & TV': 127647},http://ecx.images-amazon.com/images/I/51QWNHB3...,"{'also_bought': ['6305657947', 'B0008ENHTO', '...",NaN,"Movies & TV, Movies",Jack Weston,1996-05-03
10823,6300216217,"Elaine May wrote, directed, and starred in thi...",A New Leaf [VHS],35.00,{'Movies & TV': 235798},http://ecx.images-amazon.com/images/I/4149CFAA...,"{'also_bought': ['B00005U2K5', 'B0007QJ1ZW', '...",NaN,"Movies & TV, Movies",Jack Weston,1996-05-03
11212,6300270297,Terrence McNally adapted his Broadway farce fo...,The Ritz (1976) [VHS],34.60,{'Movies & TV': 378560},http://ecx.images-amazon.com/images/I/51RJ8PXV...,NaN,NaN,"Movies & TV, Movies",Jack Weston,1996-05-03
11233,6300270025,High Road to China is a 1983 adventure-romance...,High Road to China [VHS],24.99,{'Movies & TV': 60425},http://ecx.images-amazon.com/images/I/21G64A1H...,"{'also_bought': ['B00009AOAW', 'B001GUYAZY', '...",NaN,"Movies & TV, Movies",Jack Weston,1996-05-03
12506,6301514017,NaN,The April Fools [VHS],24.99,{'Movies & TV': 338775},http://ecx.images-amazon.com/images/I/21P0ZREQ...,NaN,NaN,"Movies & TV, Movies",Jack Weston,1996-05-03


## 5) Matching controls and meta data

In [ ]:
pd.options.mode.chained_assignment = None 
meta_control_actors = pd.DataFrame()

for idx, actor in enumerate(list(controls['Name'])):
    
    match = pd.DataFrame()
    match = actors_products[actors_products['actors'].str.contains(actor)==True].drop('actors',axis=1).drop('directors',axis=1)
    match['name'] = 'None'
    meta_control_actors = meta_control_actors.append(match)
    
    if idx %100 == 0: print('passing the {}th actor'.format(idx))

## 6) Loading Reviews data for Movies and TV and cleaning

In [6]:
datapath = 'DATA/'
filename = 'reviews_Movies_and_TV.json.gz'

In [7]:
import gzip
def gz_to_dataframe(datapath, filename):
    def parse(path): 
        g = gzip.open(path, 'rb') 
        for l in g: 
            yield eval(l) 
    def getDF(path): 
        i = 0 
        df = {} 
        for d in parse(path): 
            df[i] = d 
            i += 1 
        return pd.DataFrame.from_dict(df, orient='index') 
    return getDF(datapath+filename)
    
reviews_df = gz_to_dataframe(datapath, filename)

In [8]:
reviews_df.head()

,overall,unixReviewTime,summary,reviewText,helpful,asin,reviewTime,reviewerName,reviewerID
0,5.0,1358380800,Alton... nough said,This has some great tips as always and is help...,"[0, 0]",0000143502,"01 17, 2013",Rebecca L. Johnson,A3R5OBKS7OM2IR
1,5.0,1380672000,Ah Alton...,This is a great pastry guide. I love how Alto...,"[0, 0]",0000143529,"10 2, 2013",Rebecca L. Johnson,A3R5OBKS7OM2IR
2,2.0,1216252800,Don't waste your money,I have to admit that I am a fan of Giada's coo...,"[2, 4]",0000143561,"07 17, 2008",Great Home Cook,AH3QC2PC1VTGP
3,5.0,1236902400,VOLUME 1 & VOLUME 2-BETTER THAN THERAPY,I bought these two volumes new and spent over ...,"[9, 9]",0000143588,"03 13, 2009",Anna V. Carroll,A3LKP6WPMP9UKX
4,5.0,1232236800,Barefoot Contesst Vol 2,I am very pleased with the dvd only wish i cou...,"[1, 4]",0000143588,"01 18, 2009",Rebecca Millington,AVIY68KEPQ5ZD


In [9]:
reviews_df.shape

(4607047, 9)

In [ ]:
def clean_reviews(review_df):
    filtered_reviews = reviews_df[reviews_df['asin'].isin(list(meta_dead_actors['asin']))]
    filtered_reviews = filtered_reviews[['asin','reviewText','summary','reviewTime']]
    filtered_reviews['ReviewTime'] = list(pd.to_datetime(filtered_reviews['reviewTime'].str.replace(',','').str.replace(' ','-')))
    filtered_reviews = filtered_reviews.drop('reviewTime',axis=1)
    
    return filtered_reviews3

In [ ]:
filtered_reviews = clean_reviews(review_df)

In [22]:
filtered_reviews.set_index(['asin', 'reviewTime']).head()

reviewText  \
asin       reviewTime                                                       
0307141985 11 8, 2010   I purchased this video for my daughter, rememb...   
           11 25, 2008  I video taped this off of television for my ch...   
           11 17, 2008  "Set sail with the tiniest Pilgrim for a music...   
           11 13, 2004  I can remember every Thanksgiving me and my co...   
           04 5, 2000   I have watched &amp; enjoyed this cartoon sinc...   

                                                                summary  
asin       reviewTime                                                    
0307141985 11 8, 2010          the not so great olden days of cinema...  
           11 25, 2008                          Nostalgic Entertainment  
           11 17, 2008  Delightful Lesson In Our Nation's Early History  
           11 13, 2004              Every child would LOV E this Movie!  
           04 5, 2000                                     Mouse Forever

In [26]:
filtered_reviews.shape

(453349, 4)

## 7) Matching meta/dead data with review data

In [27]:
metactors = meta_dead_actors[['asin','name','death date', 'title','categories','salesRank']].reset_index() #.set_index(['name','death date','asin'])

In [64]:
pd.options.mode.chained_assignment = None  # default='warn'

def matching_meta_reviews(filtered_reviews, meta):
    
    processed_reviews = pd.DataFrame()
    
    for idx, asin in enumerate(meta['asin']):
        match_reviews = pd.DataFrame()
        match_reviews = filtered_reviews[filtered_reviews['asin']==asin]

        match_reviews['actor name'] = meta['name'].get_value(idx)
        match_reviews['death date'] = meta['death date'].get_value(idx)
        match_reviews['title'] = meta['title'].get_value(idx)
        match_reviews['categories'] = clean_serie(meta['categories']).get_value(idx)
        match_reviews['salesRank'] = clean_serie(meta['salesRank']).get_value(idx)

        processed_reviews = processed_reviews.append(match_reviews)

        if idx %1000 == 0: 
            print('passing the {}th asin'.format(idx))
            #break
        
    return processed_reviews

In [ ]:
processed_reviews = pd.DataFrame(matching_meta_reviews(filtered_reviews, metactors))

## 8) Matching controls and reviews

In [ ]:
metacontrols = meta_control_actors[['asin','name', 'title','categories','salesRank']].reset_index()

In [65]:
processed_reviews_ctrls = pd.DataFrame(matching_meta_reviews(filtered_reviews, metacontrols))

passing the 0th asin
passing the 1000th asin


## Save files

In [34]:
processed_reviews.to_csv('DATA/movie_reviews_matched.csv')

In [67]:
processed_reviews_ctrls.to_csv('DATA/movie_reviews_matched_ctrls.csv')